#  Présentation Problème 57 de la collection Hock-Schittkowski

Classification dans l'ouvrage : SQR-P1-1

Number of variables : n = 2

Number of constraints : 3 (1 inequality + 2 bounds)

# Modèle

$\left\{  
\begin{array}{l l l l} 
\min f(x) \\ 
\text{s.c.}\\
0.49x_2-x_1x_2-0.09 &\geq 0\\
x_1\geq 0.4,\  x_2 \geq -4
\end{array} \right.$

où $f : x \longmapsto \sum\limits_{i=1}^{44} f_i(x)^2$

avec $f_i(x) = b_i - x_1 - (0.49-x_1)\exp(-x_2(a_i-8)) \text{ pour }i=1,\ldots,44$ 

Point de départ : $x_0 = (0.42,5)$

Solution attendue : $x^* = (0.419952675,1.284845629)$

Fonction objectif à la solution : $f(x^*) =0.02845966972$

# Extension du nombre de résidus

In [1]:
include("../src/enlsip_functions.jl")

enlsip

In [2]:
n = 2
m = 16000
nb_eq = 0 # nombre de contraintes d'égalité
nb_constraints = 3 # nombre de contraintes d'égalité et d'inégalité

3

In [3]:
h = x -> 0.419952675 + (0.49-0.419952675) * exp(-1.284845629*(x-8))

#34 (generic function with 1 method)

### Données d'entrée

In [4]:
#a = [8.,8.,10.,10.,10.,10.,12.,12.,12.,12.,14.,14.,14.,16.,16.,16.,18.,18.,20.,20.,20.,22.,22.,22., 
#         24.,24.,24.,26.,26.,26.,28.,28.,30.,30.,30.,32.,32.,34.,36.,36.,38.,38.,40.,42.]

#b = [.49,.49,.48,.47,.48,.47,.46,.46,.45,.43,.45,.43,.43,.44,.43,.43,.46,.45,.42,.42,.43,.41,
#         .41,.40,.42,.40,.40,.41,.40,.41,.41,.40,.40,.40,.38,.41,.40,.40,.41,.38,.40,.40,.39,.39]

a = range(8, 42, length = m)
b = h.(a) + 0.1*randn(m)

16000-element Vector{Float64}:
 0.5967239195547385
 0.5842141193768777
 0.4575021642638575
 0.49787848076079644
 0.5460697323162458
 0.34486039494608467
 0.44835416481054824
 0.5899152031421304
 0.47342184382427643
 0.5484874800333882
 0.5077696333355003
 0.39428629463862497
 0.48851190043762127
 ⋮
 0.48319563109974006
 0.4479272561425457
 0.3036645890007969
 0.5186207464421336
 0.47101138027202677
 0.43401390045265476
 0.5905362364159419
 0.41582602393965207
 0.35367881930337675
 0.36000744843316584
 0.5818180021033068
 0.5379514568521437

In [5]:
# Résidus et matrice jacobienne associée

r_i(x::Vector,t::Float64) = x[1] + (0.49 - x[1]) * exp(-x[2]*(t - 8))
res57 = ResidualsEval(0)

function (res57::ResidualsEval)(x::Vector,rx::Vector,J::Matrix)
    # Evaluate the residuals
    if abs(res57.ctrl) == 1
        rx[:] = b - (t::Float64 -> r_i(x,t)).(a)

    # The jacobian is computed numericaly using forward differences
    # ctrl is set to 0
    elseif res57.ctrl == 2 res57.ctrl = 0 end
    return
end

In [6]:
# Contraintes et matrice jacobienne associée

cons57 = ConstraintsEval(0)

function (cons57::ConstraintsEval)(x::Vector,cx::Vector,A::Matrix)
    # Evaluate the constraints
    if abs(cons57.ctrl) == 1
        cx[:] = [0.49 * x[2] - x[1] * x[2] - 0.09, x[1] - 0.4, x[2] + 4]
    
    # The jacobian is computed anaticaly
    elseif cons57.ctrl == 2
        A[:] = [-x[2] 0.49-x[1];
        1.0 0.0;
        0.0 1.0]
    end
end

# Résolution avec ENLSIP-0.3.0

In [7]:
x0 = [0.42,5.0]
enlsip57 = enlsip(x0,res57,cons57,n,m,nb_eq,nb_constraints)

Itération 0

Itération 1

Itération 2

Itération 3

Itération 4

Itération 5

Itération 6

Itération 7

Itération 8

Itération 9

****************************************
*                                      *
*          ENLSIP-JULIA-0.3.0          *
*                                      *
****************************************

Starting point : [0.42, 5.0]

Number of equality constraints   : 0
Number of inequality constraints : 3
Constraints internal scaling     : false

iter    objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.588519e+02  0.00e+00   2.99e-01   8.030e+00   0     2   4.59e-01    0.00e+00     1.00e-01     -
   1  1.588281e+02  0.00e+00   2.37e-02   3.453e-01   0     2   1.00e+00    2.93e-01     1.00e-01     -
   2  1.588280e+02  0.00e+00   9.60e-05   3.702e-03   0     2   1.00e+00    5.86e-02     1.00e-01     -
   3  1.588280e+02  0.00e+00   8.17e-08   9.635e-04   0     2   1.00e+00    3.38e-02     1.00

ENLSIP([0.42072779988626813, 1.6602767188308756], 158.82804219664285)

In [8]:
x1_sol = enlsip57.sol[1]
x2_sol = enlsip57.sol[2]

println("Solution trouvée avec ENLSIP :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip57.obj_value  

Solution trouvée avec ENLSIP :
x_sol = (4.20727800e-01, 1.660276719e+00)
f(x_sol) = 1.588280422e+02
